In [ ]:
import numpy as np

In [ ]:
def stochastic_rounding(x):
    # takes the integer part of the number
    x_int = x.astype(np.int32)
    # takes the fractional part
    x_frac = np.abs(x - x_int)

    # generate a random number
    rng = np.random.random(x_int.shape)

    # if the frac is grater... for positive cases
    rounded_pos = np.where(x_frac > rng, x_int + 1, x_int)

    # if the grac is greate... for negative cases
    rounded_neg = np.where(x_frac > rng, x_int - 1, x_int)

    # select the rounded according to the signal
    rounded = np.where(x < 0, rounded_neg, rounded_pos)
    
    return rounded

x = np.random.normal(0, 5, (5)) 
print("float ", x)
rx = 0
it = 100
for _ in range(it):
    rx += stochastic_rounding(x)

rx = rx.astype(np.float32) / it
print("stchr0", stochastic_rounding(x))
print("stchr1", rx)
print("round ", np.around(x).astype(np.int32))

In [ ]:

def quantize(x, round_stoch = True):
    """ exponentiation and quantization function """

    # just to avoid numerical problems
    eps = 1e-8

    # extract the signal
    s = np.sign(x)

    # takes the abs
    abs_x = np.abs(x)

    cliped_abs_x = np.where(abs_x < eps, eps, abs_x) # clip the min value of abs. (this is just for avoid numercal problems)
    cliped_abs_x = np.where(cliped_abs_x > 1, 1, cliped_abs_x) # clip the max value of DN 

    # gets the exponent with base 2
    exp = np.log2(cliped_abs_x)

    # round to nearest and cast to int (use stochastic rounding)
    if round_stoch:
        round_exp = stochastic_rounding(exp)
    else:
        round_exp = (np.round(exp)).astype(np.int32)


    # stochastic zero
    
    # detect underflow
    underflow = np.where(round_exp < -7, 1, 0)

    # clip expoent in -7
    clip_exp = np.where(underflow, -7, round_exp)
    
    # randomize the signal
    s = np.where(np.logical_and(np.random.random(round_exp.shape) < 0.5, underflow), -s, s) 
    
    # convert to float32 again
    qx = s * np.power(2., clip_exp)
    return qx


for _ in range(10):
    x = np.random.normal(0, 0.2, (5)) 
    print("float ", x)
    print("quant0", quantize(x))
    print("quant1", quantize(x))
    print("quant2", quantize(x))    
    print("-------")

In [ ]:
a = np.array([0,1,0,1])
b = np.array([1,1,0,0])

print(np.logical_and(a, b))

print(np.sign(-2))

In [ ]:
# visualizing
import matplotlib.pyplot as plt


# range of values from -2,2 
float_values = np.arange(-1, 1, 0.001)


sx4_values = quantize(float_values, True)
sx4_round_values = quantize(float_values, False)


sx4_over_time = quantize(float_values, True)
for _ in range(100):
    sx4_over_time += quantize(float_values, True)
sx4_over_time /= 100


print(float_values.shape)
print(sx4_values.shape)

In [ ]:
plt.figure(figsize=(20,20), dpi=300)
plt.plot(sx4_values)
plt.plot(sx4_over_time)
plt.plot(sx4_round_values)
plt.plot(float_values)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


a = np.random.randn(784, 256) * np.sqrt(2/784)

plt.hist(np.ravel(a), bins=64)
plt.show()

In [ ]:
import quantizer
import cupy as cp

a = cp.array(2.5)
b = cp.array(4.3)
c = cp.array(a * b)
d = cp.array(quantizer.quantize_po2(a) * quantizer.quantize_po2(b))
e = cp.array(quantizer.quantize_po2(c))
f = cp.array(quantizer.quantize_po2(d))
h = quantizer.quantize_po2(a)
i = quantizer.quantize_po2(b)

print("a=%f, b%f, c%f, d%f, e%f, f%f, h%f, i%f"%(a, b, c, d, e, f, h, i))


# tests with tensorflow convolution foward and backward

In [4]:
import tensorflow as tf
import numpy as np


class CustomConvLayer():
    def __init__(self, filters, kernel_size, input_channels, strides=[1,1,1,1], padding='SAME'):        
        
        self.filters = tf.constant(np.random.randn(kernel_size, kernel_size, input_channels, filters), dtype=tf.float32)
        self.bias = tf.constant(np.random.randn(filters), dtype=tf.float32)

        # other atributes        
        self.strides = strides
        self.padding = padding

        
    def forward(self, inputs):
        self.inputs = inputs
        z = tf.nn.bias_add(tf.nn.conv2d(input=inputs, filters=self.filters, strides=self.strides, padding=self.padding), self.bias)
        return z


    def backward(self, dz, learning_rate):
        dx = tf.compat.v1.nn.conv2d_backprop_input(tf.shape(self.inputs), self.filters, dz, strides=self.strides, padding=self.padding)
        dw = tf.compat.v1.nn.conv2d_backprop_filter(self.inputs, tf.shape(self.filters), dz, strides=self.strides, padding=self.padding)
        db = tf.reduce_sum(dz, (0,1,2), True)

        # update weights
        self.filters = self.filters - learning_rate * dw
        self.bias = self.bias - learning_rate * db

        return dx
    
x = tf.random.normal((32, 416, 416, 64), dtype=tf.float32)
conv2D = CustomConvLayer(16, 3, 64)
z = conv2D.forward(x)
print(z.shape)

dx = conv2D.backward(z, 0.0001)
print(dx.shape)

(32, 416, 416, 16)
(32, 416, 416, 64)


In [ ]:
import tensorflow as tf
from dataset import load_mnist

# Exemplo de uso da camada personalizada
input_shape = (28, 28, 3)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), padding='SAME'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='SAME'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),    
    tf.keras.layers.Dense(256),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(256),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(10),        
    tf.keras.layers.Softmax()    
])



(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = tf.cast(tf.expand_dims(x_train, -1), tf.float32)
x_test = tf.cast(tf.expand_dims(x_test, -1), tf.float32)
y_train = tf.one_hot(y_train, 10, 1., 0.)
y_test = tf.one_hot(y_test, 10, 1., 0.)

# Compilação e treinamento do modelo
optimizer = tf.keras.optimizers.SGD(0.001)
batch_size = 256
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'], run_eagerly=True)
model.build([batch_size] + x_train.shape[1:])

model.fit(x_train, y_train, epochs=10, batch_size=batch_size)
